# Machine Learned Sentiment Analysis using Python

“Can our project correctly classify the sentiment of just about any
sentence in the English language?”

Our goal is to build and train a model that will be able to classify more
than one dataset with over 70% accuracy on each of them. To this end,
we might have to adjust some properties to avoid overfitting to one of the
datasets to have it perform better on a general scale. Because we are
still beginners to machine learning, we are more focused on getting a
model successfully running.


## 1.0 Binary classification using binary logistic regression

For the first part of this notebook we will build a binary classification
model, evaluated using logistic regression, as we learned in class. This
section of the project is strongly based off of this blog post by
Atharva Mashalkar.
https://towardsdatascience.com/sentiment-analysis-using-logistic-regression-and-naive-bayes-16b806eb4c4b

**Run the code cell below** to import the required packages.

To start with, we will import the libraries by nltk (Natural Language Toolkit) package, which contains 5000
which will help us with the preprocessing and training of our model. We will
also need some libraries such as regular expressions to filter out
unnecessary data.

In [1]:
import re
import string
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from nltk.corpus import twitter_samples

from sklearn.utils import shuffle

import sklearn.model_selection   # for train and test splits
import numpy as np

import nltk
nltk.download('twitter_samples')

import matplotlib.pyplot as plt
import seaborn as sns
import warnings

[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


### 1.1 Preprocessing the data
**Stop-Word Removal** : In English, words like a, an, the, as, in, on,
etc. are considered as stop-words so according to our requirements
we can remove them to reduce vocabulary size as these words don't
contribute to the final meaning or sentiment of a sentence.
To do this, we import the stopwords list from nltk. We also tokenize
each sentence  string into a list of words after cleaning up filler words
such as retweet, hashtags, and URLs.

Additionally, we need to convert all letters in the string to lowercase.
This helps reduce unnecessary bloat in the dataset.

Next, we tokenize the strings: separate all the words in the string into
a python list of words.

As a final step, we perform stemming: **stemming** refers to the process of
removing suffixes and reducing a word to some base form such that all
different variants of that word can be represented by the same form
(e.g., “walk” and “walking” are both reduced to “walk”).

In [2]:
#Preprocessing tweets
def process_tweet(tweet):
    #Remove old style retweet text "RT"
    cleaned_tweet = re.sub(r'^RT[\s]','', tweet)

    #Remove URLS
    cleaned_tweet = re.sub(r'https?:\/\/.*[\r\n]*','', cleaned_tweet)

    #Remove hashtags
    cleaned_tweet = re.sub(r'#','',cleaned_tweet)

    #convert tweet to lowercase
    cleaned_tweet = cleaned_tweet.lower()

    #Instantiate tokenizer class
    tokenizer = TweetTokenizer(preserve_case=False,    strip_handles=True, reduce_len=True)

    #Tokenize tweets
    tweet_tokens = tokenizer.tokenize(cleaned_tweet)

    #Import the english stop words list from nltk
    stopwords_english = stopwords.words('english')

    #Creating a list of words without stopwords
    tweets_clean = []
    for word in tweet_tokens:
        if word not in stopwords_english and word not in string.punctuation:
            tweets_clean.append(word)

    #Instantiate stemming class
    stemmer = PorterStemmer()

    #Creating a list of stems of words in tweet
    tweets_stem = []
    for word in tweets_clean:
        stem_word = stemmer.stem(word)
        tweets_stem.append(stem_word)

    return tweets_stem

### 1.2 Building the Frequency Dictionary

Here we define a function that will take as input tweets and their labels
as parameters. It will go through every tweet, preprocess them with the
function we just defined, count the occurrence of every word in the data
set and create a frequency dictionary.

In [3]:
#Frequency generating function
def build_freqs(tweets, ys):
    yslist = np.squeeze(ys).tolist() #squeeze is needed or the list
                                     # will end up with one element

    freqs = {}
    for y, tweet in zip(yslist, tweets):
        for word in process_tweet(tweet):
            pair = (word, y)
            freqs[pair] = freqs.get(pair, 0) + 1

    return freqs

Next, let's create a function that will extract features from a tweet
through the use of the ‘freqs’ dictionary and the defined process_tweet
function from earlier.

In [4]:
def extract_features(tweet, freqs):
    # process_tweet tokenizes, stems, and removes stopwords
    word_l = process_tweet(tweet)

    # 3 elements in the form of a 1 x 3 vector
    x = np.zeros((1, 3))

    #bias term is set to 1
    x[0,0] = 1

    # loop through each word in the list of words
    for word in word_l:

        # increment the word count for the positive label 1
        x[0,1] += freqs.get((word,1),0)

        # increment the word count for the negative label 0
        x[0,2] += freqs.get((word,0),0)

    assert(x.shape == (1, 3))
    return x

Continuing, we are importing the sample tweets from nltk and splitting
the data into training sets and test sets.

In [5]:
#import dataset from nltk
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

#create labels
one_array = [1] * len(all_positive_tweets)
zero_array = [0] * len(all_negative_tweets)

all_tweets = all_positive_tweets + all_negative_tweets
all_labels = one_array + zero_array
tweets_and_labels = np.vstack((all_tweets,all_labels)).T

#extract X (data) and y (labels) columns
nltk_X = tweets_and_labels[:,0].tolist()
nltk_y = tweets_and_labels[:,1].T.astype(int).tolist()

#split the data set into train and test sets with a 80/20 ratio
nltk_X_train, nltk_X_test, nltk_y_train, nltk_y_test = sklearn.model_selection.train_test_split(nltk_X, nltk_y, test_size=0.20, random_state=0)
nltk_y_train = np.array([nltk_y_train]).T
nltk_y_test = np.array([nltk_y_test]).T

With our required functions for processing tweets ready to go, we can begin
to build our logistic regression model.

The logistic regression maps the input $\mathbf{x}_i$ into the following
output:

$p(y_i = 1 \mid \mathbf{x}_i, \mathbf{w}) = \sigma(\mathbf{w}^T\mathbf{x}_i) =  \sigma(w_0 + w_1 x_1 + w_2 x_2)$.

$\sigma$ is the sigmoid function, that is defined as:

$\sigma(z) = \frac{1}{1 + e^{-z}} = (1+e^{-z})^{-1}$

The output of the sigmoid function is a value between 0 and 1. Let us
define the sigmoid function to be used in the model.

In [6]:
def sigmoid(z):
    """Returns the element-wise logistic sigmoid of z."""
    # Your code here. Aim for 1 line.
    return 1 / (1 + np.exp(-z))

## 1.3 Cost Function and Gradient Descent

Our goal is to find a configuration of our parameters $\mathbf{w}$ that
minimizes our objective function (BCE). For logistic regression, we use
the binary cross-entropy loss. Recall from Lecture 1 that the basic
logistic regression training objective (learning objective) is:

$$
\ell_\text{LR}(\mathbf{w}) = \sum_{i=1}^N y_i \ln \sigma(\mathbf{w}^T \mathbf{x}_i) + (1-y_i) \ln \left(1-\sigma(\mathbf{w}^T \mathbf{x}_i)\right)
$$

The "basic" gradient for the above training objective is on a slide
titled "Maximum likelihood estimate for LR" from Lecture 1, and
reproduced here:

$$
\nabla \ell_\text{LR}(\mathbf{w}) = \sum_{i=1}^N (\sigma(\mathbf{w}^T \mathbf{x}_i) - y_i)\mathbf{x}_i
$$

Let's define a few functions that implement these operations.

In [7]:
def gradient_descent(x, y, w, lr, num_iters):

    m = len(x)

    for i in range(0, num_iters):

        z = np.dot(x,w)
        h = sigmoid(z)

        # calculate the cost function
        J = (-1/m)*(np.dot(y.T,np.log(h)) + np.dot((1-y).T,np.log(1-h)))

        # update the weights by gradient descent
        w = w - (lr/m)*np.dot(x.T, h-y)

    J = float(J)
    return J, w

### 1.4 Training and Testing our Model

As all the required functions are ready we can finally train our model
using the training set.

In [8]:
nltk_freqs = build_freqs(nltk_X_train, nltk_y_train)

# collect the features 'x' and stack them into a matrix 'X'
nltk_X_gd = np.zeros((len(nltk_X_train), 3))
for i in range(len(nltk_X_train)):
    nltk_X_gd[i, :]= extract_features(nltk_X_train[i], nltk_freqs)

# training labels corresponding to X
nltk_Y_gd = nltk_y_train

# Apply gradient descent to extract the weight vector w
nltk_J, nltk_w = gradient_descent(nltk_X_gd, nltk_Y_gd, np.zeros((3, 1)), 1e-9, 1500)
print(f"The cost function at the end of training is {nltk_J:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(nltk_w)]}")

KeyboardInterrupt: 

Right above here, we receive and print the final cost J as well as the
weight matrix w after all the training is done. This weight matrix
constitutes our binary classification based on logistic regression model.

Let's proceed to write two more functions which when given a tweet, will
predict results using the freqs dictionary and weights matrix. The second
function will use the predict function and provide the accuracy of
the model on the given testing data set.

In [ ]:
def predict_tweet(tweet, freqs, theta):
    """
    Input:
        tweet: a string
        freqs: a dictionary of the frequencies of each tuple (word, label)
        theta: vector of weights
    Output:
        y_pred: the probability of a tweet being positive or negative
    """

    # extract the features of the tweet and store it into x
    x = extract_features(tweet, freqs)

    # make the prediction using x and theta
    z = np.dot(x,theta)
    y_pred = sigmoid(z)
    return y_pred

def test_logistic_regression(X_test, y_test, freqs, theta):
    """
    Input:
        X_test: a list of tweets
        y_test: vector with the corresponding labels for the list of tweets
        freqs: a dictionary of the frequencies of each tuple (word, label)
        theta: vector of weights
    Output:
        accuracy: (# of tweets classified correctly) / (total # of tweets)
    """

    # the list for storing predictions
    y_hat = []

    for tweet in X_test:
        # get the label prediction for the tweet
        y_pred = predict_tweet(tweet, freqs, theta)

        if y_pred > 0.5:
            # append 1.0 to the list
            y_hat.append(1)
        else:
            # append 0 to the list
            y_hat.append(0)
    # With the above implementation, y_hat is a list, but test_y is a
    # (m,1) array
    # convert both to one-dimensional arrays in order to compare them
    # using the '==' operator
    y_hat = np.array(y_hat)
    test_y = y_test.reshape(-1)
    accuracy = np.sum((test_y == y_hat).astype(int))/len(X_test)

    return accuracy

With all the required functions defined, we can proceed to try out our
model and look at the output.

In [ ]:
acc = test_logistic_regression(nltk_X_test, nltk_y_test, nltk_freqs, nltk_w)

print("The accuracy on the test set is {}%".format(acc*100))

We have obtained a pretty high accuracy (around 99.0%) with the trained
model. Let's formulate a few sentences and see what the model predicts
out of them.

In [ ]:
tweet = "I hate insta so much everyone is nicer on here"
prediction = predict_tweet(tweet, nltk_freqs, nltk_w)
print("This tweet is positive" if prediction >= 0.5 else "This tweet is negative")

tweet = "Everyday is my favorite, waking up one more day makes each day awesome"
prediction = predict_tweet(tweet, nltk_freqs, nltk_w)
print("This tweet is positive" if prediction >= 0.5 else "This tweet is negative")

tweet = "Checking the box full of ole' photos is always fun!"
prediction = predict_tweet(tweet, nltk_freqs, nltk_w)
print("This tweet is positive" if prediction >= 0.5 else "This tweet is negative")

### 1.6 Expanding the data set

Let's continue exploring the model by training it using another popular data
set. The data set in question is "Twitter US Airline Sentiment" (retrieved
from https://www.kaggle.com/datasets/crowdflower/twitter-airline-sentiment?resource=download ).
For the ease of use we have included the data set already
within the repository. Let's do a few lines of code to import and preview
the data in the files.

In [ ]:
df = pd.read_csv('Tweets.csv')
print(df.shape)
df.head()

Unlike our previous data set, this one contains more columns which we might
not need. It is our job to process the data to make it fit the model
defined above. However, do observe that there is one more sentiment in the
labels: the neutral label.

In [ ]:
print(df.airline_sentiment.value_counts())
plt.figure(figsize = (10, 8))
ax = sns.countplot(x = 'airline_sentiment', data = df, palette = 'pastel')
ax.set_title(label = 'Total number of sentiments of tweets', fontsize = 20)
plt.show()

Let's define a few functions that will help us with the text processing.
Remember, currently we are working with binary classification of positive
and negative sentiments. For this reason, we will be discarding all neutral
labels and their corresponding data.

In [ ]:
#remove all neutral-labeled data
df =  df[df.airline_sentiment != "neutral"]
print(df.airline_sentiment.value_counts())


Next, we should convert the sentiments into numerical values.

In [ ]:
# convert sentiments to 0 or 1
def convert_sentiment(sentiment):
    if  sentiment == "positive":
        return 1
    elif sentiment == "negative":
        return 0

df.airline_sentiment = df.airline_sentiment.apply(lambda x : convert_sentiment(x))
print(df.airline_sentiment)

One final step, we will extract each tweet text from the data and their
respective labels. We also need to split the data into training and test
sets as we did before.

In [ ]:
#extract columns in the csv file as rows
airline_X = df['text'].tolist()
airline_y = df['airline_sentiment'].tolist()

#split into 80:20 train to test ratio
airline_X_train, airline_X_test, airline_y_train, airline_y_test = sklearn.model_selection.train_test_split(airline_X, airline_y, test_size=0.20, random_state=0)
airline_y_train = np.array([airline_y_train]).T.astype(float)
airline_y_test = np.array([airline_y_test]).T.astype(float)

The data has now undergone transformation and is ready to be handled by
our previously defined functions. We simply have to repeat the code
sections written out before but now using the airline data set.

In [ ]:
airline_freqs = build_freqs(airline_X_train, airline_y_train)

# collect the features 'x' and stack them into a matrix 'X'
airline_X_gd = np.zeros((len(airline_X_train), 3))
for i in range(len(airline_X_train)):
    airline_X_gd[i, :]= extract_features(airline_X_train[i], airline_freqs)

# training labels corresponding to X
airline_Y_gd = airline_y_train

# Apply gradient descent to extract the weight vector w
airline_J, airline_w = gradient_descent(airline_X_gd, airline_Y_gd, np.zeros((3, 1)), 1e-9, 1500)
print(f"The cost function at the end of training is {airline_J:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(airline_w)]}")

airline_acc = test_logistic_regression(airline_X_test, airline_y_test, airline_freqs, airline_w)

print("The accuracy on the test set is {}%".format(airline_acc*100))

The resulting accuracy of the model trained on the airline sentiment
data seems lower than the model trained on nltk's tweets sentiments.
This is to be discussed in our report submission.

### 1.7 Combining nltk's Dataset with Twitter US Airline Sentiment Analysis's Dataset

As a final experiment, I would like to repeat the training and testing
of our model, but this time with both data sets combined. Let's do a
little processing of the data to achieve this objective.

In [ ]:
#combine both list of tweets into one data set
combined_X = airline_X + nltk_X
combined_y = airline_y + nltk_y

#split into 80:20 train to test ratio
combined_X_train, combined_X_test, combined_y_train, combined_y_test = sklearn.model_selection.train_test_split(combined_X, combined_y, test_size=0.20, random_state=0)
combined_y_train = np.array([combined_y_train]).T.astype(float)
combined_y_test = np.array([combined_y_test]).T.astype(float)

The data is now ready to go through the training loops and testing outputs.

In [ ]:
combined_freqs = build_freqs(combined_X_train, combined_y_train)

# collect the features 'x' and stack them into a matrix 'X'
combined_X_gd = np.zeros((len(combined_X_train), 3))
for i in range(len(combined_X_train)):
    combined_X_gd[i, :]= extract_features(combined_X_train[i], combined_freqs)

# training labels corresponding to X
combined_Y_gd = combined_y_train

# Apply gradient descent to extract the weight vector w
combined_J, combined_w = gradient_descent(combined_X_gd, combined_Y_gd, np.zeros((3, 1)), 1e-9, 1500)
print(f"The cost function at the end of training is {combined_J:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(combined_w)]}")

combined_acc = test_logistic_regression(combined_X_test, combined_y_test, combined_freqs, combined_w)

print("The accuracy on the test set is {}%".format(combined_acc*100))

The accuracy on this model appears to be lower than that trained solely
on nltk's sentiment data set, but shows to be higher than that trained
on the airline sentiment data set. We will discuss this in detail in the
written reports as well.

### 1.8 Training the model by carrying over the weight vector

In our previous three experiments, we conducted the training on
a clean slate by initializing the weights vectors to zero. What
would happen if we were to train our binary classification model
on one data set and pass the resulting weight vector as the
initial weight vector for the second data set? Let's take a
look!

In [ ]:
# Apply gradient descent to extract the weight vector w
shared_w_J, shared_w_w = gradient_descent(airline_X_gd, airline_Y_gd, nltk_w, 1e-9, 1500)
print(f"The cost function at the end of training is {shared_w_J:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(shared_w_w)]}")

Because the model has already been trained on both data sets,
we are testing it against the combined set's test set.

In [ ]:
shared_w_acc = test_logistic_regression(combined_X_test, combined_y_test, combined_freqs, shared_w_w)

print("The accuracy on the test set is {}%".format(shared_w_acc*100))

The results are close enough to that of the combined data set's model's
results. As usual, we will discuss the significance of this output
inside the written report.

### 1.9 Introducing one more dataset: Sentiment140

We will perform sections 1.6 to 1.10 again with an additional dataset.
The sentiment140 dataset contains 1,600,000 tweets extracted using the
Twitter API (retrieved from https://www.kaggle.com/datasets/kazanova/sentiment140 ).
This means we are dealing with tweets once more! The tweets
have been labeled (0 for negative, 2 for neutral, 4 for positive).

In [ ]:
df_sentiment140 = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding='latin-1', header=None)
df_sentiment140.columns = ["target", "id", "date", "flag", "user", "text"]
print(df_sentiment140.shape)
df_sentiment140.head()


This dataset is actually over 200 MB in file size and way too large to
comfortably work with on a low-end computer. We will only work with
about half of the data, shuffled with a constant random_state variable so
that it is reproducible every time.

In [ ]:
df_sentiment140 = shuffle(df_sentiment140,random_state=0)
df_sentiment140 = df_sentiment140[1:500000]
df_sentiment140['target'].value_counts()

Like the airline sentiment data set, this one contains some columns which
we might not need. We will once again have to preprocess the data. The
neutral label is also present and will have to be ignored.

In [ ]:
print(df_sentiment140.target.value_counts())
plt.figure(figsize = (10, 8))
ax = sns.countplot(x = 'target', data = df_sentiment140, palette = 'pastel')
ax.set_title(label = 'Total number of sentiments of tweets', fontsize = 20)
plt.show()

In [ ]:
#remove all neutral-labeled data
df_sentiment140 =  df_sentiment140[df_sentiment140.target != 2]
print(df_sentiment140.target.value_counts())


Next, we should convert the sentiments into numerical values.

In [ ]:
# convert sentiments to 0 or 1
def convert_sentiment_140(sentiment):
    if  sentiment == 4:
        return 1
    elif sentiment == 0:
        return 0

df_sentiment140.target = df_sentiment140.target.apply(lambda x : convert_sentiment_140(x))
print(df_sentiment140.target)

Once again, we need to extract the data and their corresponding labels. 
We also need to split the data into training and test sets as we did before.

In [ ]:
#extract columns in the csv file as rows
sentiment140_X = df_sentiment140['text'].tolist()
sentiment140_y = df_sentiment140['target'].tolist()

#split into 80:20 train to test ratio
sentiment140_X_train, sentiment140_X_test, sentiment140_y_train, sentiment140_y_test = sklearn.model_selection.train_test_split(sentiment140_X, sentiment140_y, test_size=0.20, random_state=0)
sentiment140_y_train = np.array([sentiment140_y_train]).T.astype(float)
sentiment140_y_test = np.array([sentiment140_y_test]).T.astype(float)

The data has been successfully extracted and is now ready to be handled by
the simple logistic regression model. We simply have to repeat the code
sections written out before but now using the sentiment140 data set.

In [ ]:
sentiment140_freqs = build_freqs(sentiment140_X_train, sentiment140_y_train)

# collect the features 'x' and stack them into a matrix 'X'
sentiment140_X_gd = np.zeros((len(sentiment140_X_train), 3))
for i in range(len(sentiment140_X_train)):
    sentiment140_X_gd[i, :]= extract_features(sentiment140_X_train[i], sentiment140_freqs)

# training labels corresponding to X
sentiment140_Y_gd = sentiment140_y_train

# Apply gradient descent to extract the weight vector w
sentiment140_J, sentiment140_w = gradient_descent(sentiment140_X_gd, sentiment140_Y_gd, np.zeros((3, 1)), 1e-9, 1500)
print(f"The cost function at the end of training is {sentiment140_J:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(sentiment140_w)]}")

sentiment140_acc = test_logistic_regression(sentiment140_X_test, sentiment140_y_test, sentiment140_freqs, sentiment140_w)

print("The accuracy on the test set is {}%".format(sentiment140_acc*100))

Taking a look at the results of the model trained on the sentiment140 dataset,
we can observe that it is even lower than that of the airline sentiment
dataset's! This is to be discussed in our report submission.

### 1.10 Training the model using all three datasets

One final experiment to end this section: we will carry over the weights
obtained from training the model on sentiment140's dataset and continue
the training with the combined dataset of nltk and airline sentiments.

In [ ]:
# Apply gradient descent to extract the weight vector w
reshared_w_J, reshared_w_w = gradient_descent(combined_X_gd, combined_Y_gd, sentiment140_w, 1e-9, 1500)
print(f"The cost function at the end of training is {reshared_w_J:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(reshared_w_w)]}")

In [ ]:
combined_freqs.update(sentiment140_freqs)

reshared_w_acc = test_logistic_regression(combined_X_test, combined_y_test, combined_freqs, reshared_w_w)

print("The accuracy on the test set is {}%".format(reshared_w_acc*100))

With one quick look at the accuracy, we can see that it has greatly
improved. The reasons for this will be discussed within the report.
This concludes all our experiments and the section for sentiment analysis
by binary classification using logistic regression.